In [1]:
import tensorflow as tf
import numpy as np
import PIL
from PIL import Image

@tf.function
def get_rotation_matrix(rad, height, width):
    bx_s = float(width // 2)
    by_s = float(height // 2)
    # 先垂直反射, 然后左移bx_s, 下移by_s，（bx_s, by_s）移动到了左上原点的地方
    T_s_d1 = tf.stack([
        tf.concat([tf.constant([1.0]), tf.constant([0.0]), tf.constant([-bx_s]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([-1.0]), tf.constant([by_s]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([0.0]), tf.constant([1.0]) ], axis = 0)
        ], axis = 0)
    # 旋转
    T_d1_d2 = tf.stack([
        tf.concat([tf.cos([rad]), tf.sin([rad]), tf.constant([0.0]) ], axis = 0),
        tf.concat([-tf.sin([rad]), tf.cos([rad]), tf.constant([0.0]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([0.0]), tf.constant([1.0]) ], axis = 0)
        ], axis = 0)
    # 先再度y轴反射， 然后右移bx_s，下移by_s
    T_d2_d = tf.stack([
        tf.concat([tf.constant([1.0]), tf.constant([0.0]), tf.constant([bx_s]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([-1.0]), tf.constant([by_s]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([0.0]), tf.constant([1.0]) ], axis = 0)
        ], axis = 0)
    rotation_mat =  T_d2_d @ T_d1_d2 @ T_s_d1
    return rotation_mat


@tf.function
def warpAffine(image, matrix_inv, width, height):
    old_height, old_width, channel_num = image.shape
    # dst = tf.Variable(tf.zeros((height, width, channel_num), dtype=tf.float32), trainable=False)
    
    i = tf.constant(0)
    c = lambda i: tf.less(i, width)

    def loop_width(i):
        tf.print(i)
        return (tf.add(i, 1),)
    r = tf.while_loop(c, loop_width, [i])
    return r

    """
    matrix: inverse matrix, i.e., map from dest to source
    width: width of destination
    height: height of destination
    """
    # old_height, old_width, channel_num = image.shape
    # # dst = tf.Variable(tf.zeros((height, width, channel_num), dtype=tf.float32), trainable=False)
    # def do(i):
    #     tf.print(i)
    #     return (tf.add(i, 1),)
    # tf.while_loop(
    #     lambda i: tf.less(i, width),
    #     do,
    #     [tf.constant(0)]
    # )
    # for u in range(width):
    #     for v in range(height):
    #         tf.print(u)
            # x, y, _ = matrix_inv @ tf.constant([ [u], [v], [1.0] ], dtype=tf.float32)
            # x, y, = x[0], y[0]
            # intx, inty = int(x), int(y)
            # if (0 <= intx < old_width and 0 <= inty < old_height):
            #     pix = image[inty, intx]
            #     dst[v, u].assign(pix)
    # return dst

@tf.function
def rotate_image(image, degree):
    rad = degree * np.pi / 180
    height, width, _ = image.shape
    
    rotate_matrix = get_rotation_matrix(rad, height, width)
    rotate_matrix_inv = tf.linalg.inv(rotate_matrix)
    dest_img = warpAffine(image, rotate_matrix_inv, width, height)
    return dest_img

In [5]:
tf.meshgrid(tf.range(10), tf.range(5))

[<tf.Tensor: shape=(5, 10), dtype=int32, numpy=
 array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=int32)>,
 <tf.Tensor: shape=(5, 10), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]], dtype=int32)>]

In [6]:
x = [1, 2, 3]
y = [4, 5, 6]
X, Y = tf.meshgrid(x, y)

In [10]:
X.value_index

AttributeError: Tensor.value_index is meaningless when eager execution is enabled.

: 

In [9]:
dir(X)

['OVERLOADABLE_OPERATORS',
 '_USE_EQUALITY',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__complex__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__iter__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmatmul__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',


In [2]:
tf.add(5, 1)

2022-07-05 12:44:07.664342: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-05 12:44:07.664472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-05 12:44:07.743529: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-05 12:44:07.743656: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-05 12:44:07.743743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

<tf.Tensor: shape=(), dtype=int32, numpy=6>

In [3]:
img = Image.open('demo.jpeg')

In [4]:
res = rotate_image(np.asarray(img), 45)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [94]:
i = tf.constant(0)
c = lambda i: tf.less(i, 10)

def b(i):
    tf.print(i)
    return (tf.add(i, 1),)
r = tf.while_loop(c, b, [i])


0
1
2
3
4
5
6
7
8
9


In [95]:
i

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [88]:
i

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [1]:
2 * 2 + 4 * 3 + 2 * 4

24